In [1]:
import spacy 
nlp = spacy.load("en")
from pymongo import MongoClient
import pandas as pd

In [2]:
client = MongoClient()
db =  client.beer_recipes
coll = db.beer_recipes_abv
print(f"The database has {coll.estimated_document_count()} documents.")

The database has 436 documents.


In [3]:
df = pd.DataFrame(list(coll.find()))
df.head(1)

,ABV,IBU,SRM,_id,date,ingredients,ingredients_spacy,ingredients_stemmed,instructions,keywords,recipe,specifications,style
0,5.12,20.7,3.6,5d0aa9af52ab1402341a4b0b,2019-06-19 21:31:27.951,11.0 lb (4.99 kg) Briess two-row malt\n0.75 oz...,11.0 ( 4.99 kg ) two - row malt 0.75 ( 21 g ...,"[11.0, 4.99, two-row, malt, 0.75, 21, willamet...",\nMash grains at 154° F (68° C) for 60 minutes...,"[5.12, 20.7, 3.6, Cream Ale]",$12 Cream Ale,\nOriginalGravity:1.054FinalGravity:1.015 ABV ...,Cream Ale


In [4]:
styles_list=db["beer_recipes_abv"].distinct("style")
styles_list

['Cream Ale',
 'Belgian and French Ale',
 'Scottish and Irish Ale',
 'European Amber Lager',
 'Classic American Pilsner',
 'India Pale Ale',
 'Specialty Beer',
 'Schwarzbier',
 'German Wheat and Rye Beer',
 'Amber Hybrid Beer',
 'Stout',
 'Weizenbock',
 'Baltic Porter',
 'Oatmeal Stout',
 'American IPA',
 'American Brown Ale',
 'American Pale Ale',
 'Blonde Ale',
 'Light Lager',
 'American Barleywine',
 'English Brown Ale',
 'Robust Porter',
 'Brown Porter',
 'Saison',
 'Scottish Export 80/-',
 'English Pale Ale',
 'Belgian Specialty Ale',
 'Oktoberfest/Märzen',
 'Belgian Dark Strong Ale',
 'Other Smoked Beer',
 'American Ale',
 'Sweet Stout',
 'Pilsner',
 'Spice/Herb/Vegetable Beer',
 'Special/Best/Premium Bitter',
 'Irish Red Ale',
 'Witbier',
 'English Porter',
 'Weizen/Weissbier',
 'Russian Imperial Stout',
 'Fruit Beer',
 'Doppelbock',
 'Munich Dunkel',
 'American Amber Ale',
 'Belgian Tripel',
 'Gueuze',
 'English Barleywine',
 'Dark Lager',
 'Wood-Aged Beer',
 'Extra Special/Str

In [5]:
df_eda = pd.DataFrame(list(coll.find()))
df_eda['style'].value_counts()

American IPA                              36
Specialty Beer                            25
Spice/Herb/Vegetable Beer                 17
Saison                                    17
American Pale Ale                         15
Fruit Beer                                12
English Brown Ale                         10
Belgian Specialty Ale                      9
Weizen/Weissbier                           9
Oktoberfest/Märzen                         8
American Barleywine                        8
Blonde Ale                                 8
Belgian and French Ale                     7
Russian Imperial Stout                     7
Baltic Porter                              7
India Pale Ale                             7
Extra Special/Strong Bitter                6
Amber Hybrid Beer                          6
Brown Porter                               6
American Ale                               6
Imperial IPA                               6
Light Lager                                6
English Ba

In [6]:
recipe_list=[]
for x in coll.find({}, {"recipe": 1 , "ingredients_spacy": 1, "_id": 0}):
   recipe_list.append((x['recipe'], str(x['ingredients_spacy']).strip('[]')))
labels = ['recipe', 'ingredients_spacy']
df = pd.DataFrame.from_records(recipe_list, columns=labels)
df.info(),

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 436 entries, 0 to 435
Data columns (total 2 columns):
recipe               436 non-null object
ingredients_spacy    436 non-null object
dtypes: object(2)
memory usage: 6.9+ KB


(None,)

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english')
# tfidf_matrix = tfidf.fit_transform(df[['specifications','style']])
tfidf_matrix = tfidf.fit_transform(df.ingredients_spacy)
tfidf_matrix.shape

(436, 1592)

In [8]:
from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_sim.shape

(436, 436)

In [17]:
indices = pd.Series(df.index, index=df['recipe']).drop_duplicates()
for row in sorted(indices.iteritems()):
    print(row)

('#BiggestSmallBeerEver Porter', 27)
('$12 Cream Ale', 0)
('1834 Augsburger Lagerbier', 160)
('1879 Kulmbacher Export', 209)
('2010 Barleywine', 302)
('2015 Big Brew Recipe #2: Old School Barleywine', 23)
('2015 Big Brew Recipe #3: Killer Kölsch', 396)
('7k Czech Pale Lager', 22)
('80 Shilling Scottish Ale', 29)
('A Festbier For the Rest Of Us', 35)
('AG24 Oktoberfest', 59)
('Abbey Saison', 44)
('Adroit Theory Triptych Tart Rambutan Saison', 54)
('Alabama Hopslammer American Pale Ale', 62)
('Albanach Láidir', 75)
('Ale of Asgard', 84)
('Allagash White Clone', 89)
('Amahl’s English Brown Ale', 102)
('Amber Waves of Grain', 115)
('American IPA con Pomelo Rosado (Pink Grapefruit American IPA)', 129)
('American Malt Liquor (Olde English Clone)', 132)
('American-Style Cascadian India Dark Double Black IPA (ASCIDDBIPA)', 139)
('Anchor Brewing Old Foghorn', 153)
('Anchor Steam Clone', 161)
('Ardley Brown Ale', 187)
('Ass Kicking Cream Ale (Tequila-aged Lime/Jalapeno Ale)', 199)
('Atlas Brew W

In [10]:
def get_recommendations(recipe, cosine_sim=cosine_sim):
    
    idx = indices[recipe]
    sim_scores = list(enumerate(cosine_sim[idx]))
    if len(sim_scores) >= 5:
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        sim_scores = sim_scores[1:6]
    else:
        return('No Matches')
    beer_indices = [i[0] for i in sim_scores]
    beer_scores = [i[1] for i in sim_scores]
    return df['recipe'].iloc[beer_indices], beer_scores

In [12]:
get_recommendations('Bell’s Two Hearted Ale Clone')

(255                                          Neural Rust
 423                                   West Coast Blaster
 351                   Sippin’ on the Dock of the Bay IPA
 265    North Coast Old Rasputin Russian Imperial Stou...
 0                                          $12 Cream Ale
 Name: recipe, dtype: object,
 [0.5217012363864234,
  0.46842736318260264,
  0.4646940112634247,
  0.4080110719135224,
  0.36291077492574364])

In [13]:
get_recommendations('Bill’s Farmhouse Ale')

(270                Oatmeal Belgian Pale Ale
 167                      Harvest Ale Saison
 169                       HBock Helles Bock
 101    Denizens Brewing Co. Lowest Lord ESB
 92                             Dan’s Märzen
 Name: recipe, dtype: object,
 [0.467803419392212,
  0.412662232484136,
  0.3809872989951339,
  0.3683856443404432,
  0.3560729653042686])

In [14]:
get_recommendations('Big Brew 2013 Recipe: Pointon Proper')

(247                  Mister Manners
 52          Brighid’s Irish Red Ale
 275              Old Ball and Chain
 435    I’m Not Bitter, I’m Thirsty!
 368           Carla’s Oat Brown Ale
 Name: recipe, dtype: object,
 [0.28884092318529747,
  0.2591923190294069,
  0.2578003858039055,
  0.2544338261781113,
  0.2480145893519604])

In [18]:
get_recommendations('Allagash White Clone')

(55                            Brunchmaster 2000 Witbier
 291    Pedal Haus Brewery Biere Blanche Belgian Witbier
 221                  Lawson’s Sip of Sunshine IPA Clone
 424                              Hoegaarden White Clone
 268                            Nuclear Winter White IPA
 Name: recipe, dtype: object,
 [0.41403106829212855,
  0.39906358345498494,
  0.37686465731787033,
  0.37044933240463784,
  0.30692103242897784])